In [1]:
import pickle
from pathlib import Path

import igl
import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import construction as ulac


In [2]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = {"MV": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [5]:
with segmentation_file_name.open("rb") as f:
    segmentation_paths = pickle.load(f)
markers = ulac.get_markers(segmentation_paths)
parameterized_paths = ulac.parameterize_paths(mesh, segmentation_paths, markers)
uac_paths = ulac.construct_uac_paths(parameterized_paths, markers)

In [7]:
plotter = pv.Plotter(window_size=[700, 700])
plotter.add_mesh(mesh, style="wireframe", color="grey")
point_mesh = pv.PolyData(mesh.points[boundary_inds])
point_mesh.point_data["alpha"] = boundary_values[:, 0]
point_mesh.point_data["beta"] = boundary_values[:, 1]
plotter.add_mesh(point_mesh, scalars="alpha", render_points_as_spheres=True, point_size=10)
plotter.show()

Widget(value='<iframe src="http://localhost:43329/index.html?ui=P_0x7f291bd0ac10_1&reconnect=auto" class="pyvi…

In [ ]:
mapping_3d = np.hstack((harmonic_map, np.zeros((harmonic_map.shape[0], 1))))
uac_mesh = pv.PolyData(mapping_3d, mesh.faces)

plotter = pv.Plotter(window_size=[700, 700])
plotter.add_mesh(uac_mesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()